<a href="https://colab.research.google.com/github/rahulbhatt9/Sports-Projects/blob/main/MLB_HOF_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading Packages

In [415]:
# Data Processing
import pandas as pd
import numpy as np

# Modelling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, ConfusionMatrixDisplay
from sklearn.model_selection import RandomizedSearchCV, train_test_split
from scipy.stats import randint

# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

# Loading/Cleaning Data

In [416]:
df_batting = pd.read_csv('Batting.csv')
df_batting = df_batting.groupby('playerID').sum(numeric_only=True)
df_batting = df_batting.drop('stint', axis=1)
df_batting

,yearID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
playerID,,,,,,,,,,,,,,,,,,
aardsda01,18084,331,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,1.0,0.0,0.0
aaronha01,45195,3298,12364,2174,3771,624,98,755,2297.0,240.0,73.0,1402,1383.0,293.0,32.0,21.0,121.0,328.0
aaronto01,13768,437,944,102,216,42,6,13,94.0,9.0,8.0,86,145.0,3.0,0.0,9.0,6.0,36.0
aasedo01,25786,448,5,0,0,0,0,0,0.0,0.0,0.0,0,3.0,0.0,0.0,0.0,0.0,0.0
abadan01,6010,15,21,1,2,0,0,0,0.0,0.0,1.0,4,5.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zupofr01,5876,16,18,3,3,1,0,0,0.0,0.0,0.0,2,6.0,0.0,0.0,0.0,0.0,0.0
zuvelpa01,17875,209,491,41,109,17,2,2,20.0,2.0,0.0,34,50.0,1.0,2.0,18.0,0.0,8.0
zuverge01,19551,266,142,5,21,2,1,0,7.0,0.0,1.0,9,39.0,0.0,0.0,16.0,0.0,3.0


In [417]:
df_fielding = pd.read_csv('Fielding.csv')
df_fielding = df_fielding.groupby('playerID').sum(numeric_only=True)
df_fielding

,yearID,stint,G,GS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR
playerID,,,,,,,,,,,,,,
aardsda01,18084,9,331,0.0,1011.0,11,29,3.0,2,0.0,0.0,0.0,0.0,0.0
aaronha01,70739,36,3020,2977.0,78414.0,7436,429,144.0,218,0.0,0.0,0.0,0.0,0.0
aaronto01,35389,18,387,206.0,6472.0,1317,113,22.0,124,0.0,0.0,0.0,0.0,0.0
aasedo01,25786,13,448,91.0,3328.0,67,135,13.0,10,0.0,0.0,0.0,0.0,0.0
abadan01,6007,3,9,4.0,138.0,37,1,1.0,3,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
zupofr01,5876,3,13,1.0,114.0,31,1,2.0,0,1.0,1.0,2.0,1.0,0.0
zuvelpa01,29792,15,201,136.0,3844.0,267,415,23.0,84,0.0,0.0,0.0,0.0,0.0
zuverge01,19551,12,265,31.0,1847.0,45,145,7.0,10,0.0,0.0,0.0,0.0,0.0


In [418]:
df_HOF = pd.read_csv('HallOfFame.csv')
df_HOF = df_HOF[ (df_HOF['votedBy'] == 'BBWAA') & (df_HOF['inducted'] == 'Y') & (df_HOF['category'] == 'Player') ].reset_index()
df_HOF = df_HOF[['playerID', 'inducted']]
df_HOF

,playerID,inducted
0,cobbty01,Y
1,ruthba01,Y
2,wagneho01,Y
3,mathech01,Y
4,johnswa01,Y
...,...,...
125,martied01,Y
126,mussimi01,Y
127,jeterde01,Y
128,walkela01,Y


In [419]:
df = pd.merge(df_batting, df_HOF, on='playerID', how='outer')
df = df.drop('yearID', axis=1)
df['inducted'] = df['inducted'].replace("Y", 1)
df = df.fillna(0)
df = df.sample(frac=1)
df

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,inducted
7382,hafeych01,1283,4625,777,1466,341,67,164,833.0,70.0,7.0,372,477.0,0.0,33.0,83.0,0.0,28.0,0.0
9207,johnsvi01,42,42,2,5,1,0,0,3.0,0.0,0.0,0,6.0,0.0,0.0,0.0,0.0,0.0,0.0
117,adkinhe01,1,5,1,2,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
493,ariasru01,34,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
2384,burgmto01,782,66,20,14,2,0,0,0.0,1.0,0.0,3,20.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13339,neumeda01,3,1,0,0,0,0,0,0.0,0.0,0.0,0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
9776,kingsge01,211,533,65,134,17,5,3,53.0,15.0,11.0,39,96.0,0.0,6.0,13.0,5.0,16.0,0.0
1948,brandja01,1221,3895,540,1020,175,37,112,485.0,45.0,30.0,351,574.0,20.0,12.0,31.0,29.0,75.0,0.0
4462,delahfr01,287,986,109,223,22,22,5,94.0,50.0,0.0,66,78.0,0.0,7.0,33.0,0.0,0.0,0.0


# Making Machine Learning Model

In [420]:
# Split the data into features (X) and target (y)
features = df.drop(['playerID', 'inducted'], axis=1)
target = df['inducted']
player_ids = df['playerID']

# Split the data into training and test sets
features_train, features_test, target_train, target_test, player_ids_train, player_ids_test = train_test_split(features, target, player_ids, test_size=0.5)

In [421]:
rf = RandomForestClassifier()
rf.fit(features_train, target_train)

RandomForestClassifier()

In [422]:
target_pred = rf.predict(features_test)

# Looking at Results

In [423]:
accuracy = accuracy_score(target_test, target_pred)
print("Accuracy:", accuracy)

Accuracy: 0.9934538348803127


In [424]:
features_test['inducted'] = target_test
features_test['inducted_pred'] = target_pred
features_test.insert(0, 'playerID', player_ids_test)
features_test

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,inducted,inducted_pred
18168,tavarje01,228,423,63,101,12,3,3,33.0,13.0,8.0,30,62.0,1.0,1.0,7.0,2.0,5.0,0.0,0.0
16804,sheehji01,1,4,0,0,0,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19625,whisema01,190,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10995,lugoru01,102,0,0,0,0,0,0,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7924,heintto01,90,140,17,34,5,0,3,12.0,0.0,0.0,14,22.0,2.0,0.0,1.0,0.0,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12786,moorete01,1298,4700,719,1318,263,28,80,513.0,82.0,0.0,406,368.0,0.0,16.0,82.0,0.0,76.0,0.0,0.0
1076,beamal01,2,6,0,1,1,0,0,0.0,0.0,0.0,0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7497,hamiljo02,243,339,19,43,8,1,4,22.0,0.0,0.0,8,167.0,0.0,0.0,36.0,5.0,2.0,0.0,0.0
13995,palldo01,328,5,0,0,0,0,0,0.0,0.0,0.0,1,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [428]:
features_test = features_test.sort_values('inducted_pred', ascending=False)
features_test.head(25)

,playerID,G,AB,R,H,2B,3B,HR,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP,inducted,inducted_pred
3417,cobbty01,3035,11436,2247,4189,724,295,117,1944.0,896.0,178.0,1249,608.0,0.0,94.0,292.0,0.0,0.0,1.0,1.0
11720,mayswi01,2992,10881,2062,3283,523,140,660,1903.0,338.0,103.0,1464,1526.0,192.0,44.0,13.0,91.0,251.0,1.0,1.0
16465,schmimi01,2404,8352,1506,2234,408,59,548,1595.0,174.0,92.0,1507,1883.0,201.0,79.0,16.0,108.0,156.0,1.0,1.0
14881,pujolal01,3080,11421,1914,3384,686,16,703,2218.0,117.0,43.0,1373,1404.0,316.0,123.0,1.0,123.0,426.0,0.0,1.0
7231,grovele01,619,1369,119,202,27,5,15,121.0,1.0,1.0,105,593.0,0.0,2.0,103.0,0.0,1.0,1.0,1.0
19148,wagneho01,2794,10439,1739,3420,643,252,101,1733.0,723.0,15.0,963,735.0,0.0,125.0,221.0,0.0,0.0,1.0,1.0
19878,willite01,2292,7706,1798,2654,525,71,521,1839.0,24.0,17.0,2021,709.0,86.0,39.0,5.0,20.0,197.0,1.0,1.0
13141,murraed02,3026,11336,1627,3255,560,35,504,1917.0,110.0,43.0,1333,1516.0,222.0,18.0,2.0,128.0,315.0,1.0,1.0
16061,ruthba01,2503,8398,2174,2873,506,136,714,2217.0,123.0,117.0,2062,1330.0,0.0,43.0,113.0,0.0,2.0,1.0,1.0
20364,yountro01,2856,11008,1632,3142,583,126,251,1406.0,271.0,105.0,966,1350.0,95.0,48.0,104.0,123.0,217.0,1.0,1.0
